In [1]:
!pip install gym==0.24.0
!pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.4/694.4 kB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for gym: filename=gym-0.24.0-py3-none-any.whl size=790675 sha256=b46322e7ce9c528c1ad002d6ecaf30d6c753ddc1a80ee8e6eae0d41c2042989c
  Stored in directory: /root/.cache/pip/wheels/2f/a1/b1/5f4110c8943b36e6cdfcc0e5768481639fa042b3c8357e82cc
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.26.2
    Uninstalling gym-0.26.2:
      Successfully uninstalled gym-0.26.2


In [2]:
pip install keras-rl2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip show gym

Name: gym
Version: 0.24.0
Summary: Gym: A universal API for reinforcement learning environments
Home-page: https://www.gymlibrary.ml/
Author: Gym Community
Author-email: jkterry@umd.edu
License: MIT
Location: /opt/conda/lib/python3.10/site-packages
Requires: cloudpickle, gym-notices, numpy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
import gym
import random
import time
from collections import deque, namedtuple
!pip install tensorflow==2.3.0

import numpy as np
import PIL.Image
import tensorflow as tf
import utils
!pip install pyvirtualdisplay
from pyvirtualdisplay import Display
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import MSE
from tensorflow.keras.optimizers import Adam
from gym.wrappers.record_video import RecordVideo

ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.0 (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0)
ERROR: No matching distribution found for tensorflow==2.3.0


In [5]:
env = gym.make('CartPole-v0') #Building Environmnet
states=env.observation_space.shape[0] #Getting states and actions 
actions=env.action_space.n

/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:568: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [6]:
print(states,actions)

4 2


In [7]:
pip install gym[classic_control]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 46.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install pygame 

import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

<Surface(640x480x32 SW)>

In [9]:
s=[]
episodes=10

env = RecordVideo(env, '/kaggle/working/before.mp4')

for episode in range(1,episodes+1):
    state=env.reset() #resetting the env
    #terminated=False
    done=False
    score=0
    
    while not done:
        #PIL.Image.fromarray(env.render(mode='rgb_array'))
        env.render()
        action=random.choice([0,1]) #0 left 1 right ig
        n_state,reward,done,info=env.step(action) 
        score+=reward
        s.append(score)
    print('Episode:{} Score:{}'.format(episode,score))

Episode:1 Score:17.0
Episode:2 Score:23.0
Episode:3 Score:21.0
Episode:4 Score:25.0
Episode:5 Score:13.0
Episode:6 Score:17.0
Episode:7 Score:19.0
Episode:8 Score:14.0
Episode:9 Score:44.0
Episode:10 Score:16.0


In [10]:
print(max(s))

44.0


In [11]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers.legacy import Adam

In [12]:
def build_model(states , actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model

In [13]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [14]:
model=build_model(states,actions)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 24)                120       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [15]:
def build_agent(model,actions):
    policy = BoltzmannQPolicy()
    memory=SequentialMemory(limit=50000 , window_length=1)
    dqn = DQNAgent(model=model , memory=memory ,policy=policy , nb_actions=actions , nb_steps_warmup =10 , target_model_update = 1e-2)
    return dqn

In [16]:
dqn=build_agent(model,actions)
dqn.compile(Adam(lr=1e-3),metrics=['mae'])
dqn.fit(env,nb_steps=50000,visualize=False , verbose=1)

/opt/conda/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 13:28 - reward: 1.0000

/opt/conda/lib/python3.10/site-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/opt/conda/lib/python3.10/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/opt/conda/lib/python3.10/site-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   34/10000 [..............................] - ETA: 3:37 - reward: 1.0000

/opt/conda/lib/python3.10/site-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/opt/conda/lib/python3.10/site-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/opt/conda/lib/python3.10/site-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/opt/conda/lib/python3.10/site-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/opt/conda/lib/python3.10/site-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) instead
  batch

10000/10000 [==============================] - 68s 7ms/step - reward: 1.0000
103 episodes - episode_reward: 97.000 [9.000, 200.000] - loss: 3.373 - mae: 19.068 - mean_q: 38.535

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 70s 7ms/step - reward: 1.0000
50 episodes - episode_reward: 198.540 [184.000, 200.000] - loss: 7.531 - mae: 40.271 - mean_q: 81.226

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 70s 7ms/step - reward: 1.0000
50 episodes - episode_reward: 199.760 [195.000, 200.000] - loss: 6.363 - mae: 43.414 - mean_q: 87.348

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 72s 7ms/step - reward: 1.0000
50 episodes - episode_reward: 200.000 [200.000, 200.000] - loss: 6.722 - mae: 43.232 - mean_q: 86.915

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 73s 7ms/step - reward: 1.0000
done, took 354.352 seconds


In [17]:
scores=dqn.test(env,nb_episodes=100,visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [18]:

env = RecordVideo(env, '')
_=dqn.test(env,nb_episodes=5,visualize=True)

/opt/conda/lib/python3.10/site-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /kaggle/working folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
